In [1]:
using LibExpat

### Read and parse the XML file

In [2]:
route_xml_path = "map.rou.xml"

xml_str = readstring(route_xml_path);
etree = xp_parse(xml_str);

routes = filter(e -> typeof(e) == LibExpat.ETree, etree.elements)
routes = map(route -> filter(e -> typeof(e) == LibExpat.ETree, route.elements)[1], routes);

In [3]:
function parse_each_egde_str(edge_str)
    
    sign = '+'
    edge_id_start = 1
    
    if edge_str[1] == '-'
        sign = '-'
        edge_id_start = 2
    end
    
    hashtag_idx = find(d -> d == '#', edge_str)
    if length(hashtag_idx) != 0
        edge_id_end = hashtag_idx[1]
    else
        edge_id_end = length(edge_str)
    end

    return (sign, parse(edge_str[edge_id_start:edge_id_end]), parse(edge_str[edge_id_end+1:end]))
end

parse_each_egde_str (generic function with 1 method)

### Extract all routes

In [4]:
paths = map(route -> map(s -> parse_each_egde_str(s), split(route.attr["edges"], " ")), routes);

### Turn all route edges into nodes

In [5]:
using LightGraphs
import LightGraphs.SimpleGraphs: SimpleEdge, SimpleDiGraph
include("CreateOSMGraphs.jl")
using CreateOSMGraphs;
include("utils.jl")

# osm_fnX= "northern_stockholm.osm"
osm_fn = "map.osm"
@time graph, vprops, eprops, edgeDict, 
nodesLLA, highways, geohash2edgedict = CreateOSMGraphs.CreateOSMGraph(osm_fn);

getOSMData



Use "const Styles = @compat Union{Style,Dict{Int,Style}}" instead.


  2.304198 seconds (3.38 M allocations: 154.308 MiB, 5.60% gc time)
intersections
  0.000191 seconds (1.11 k allocations: 120.531 KiB)
roadways
  0.035585 seconds (1.90 k allocations: 109.782 KiB)
segmentHighways
  0.676336 seconds (89.79 k allocations: 4.646 MiB)
createGraph
  0.484611 seconds (89.39 k allocations: 3.865 MiB)
  8.061682 seconds (4.38 M allocations: 206.784 MiB, 2.19% gc time)


In [16]:
for path in paths
    for edge in path
        @assert haskey(highways, edge[2])
        nodes = highways[edge[2]].nodes
        @assert length(nodes) >= 2
        @assert haskey(vprops, nodes[1])
        @assert haskey(vprops, nodes[end])
    end
end

In [23]:
highways[552009977].nodes

9-element Array{Int64,1}:
  259527614
 3121765161
  446665330
  937553017
  446665335
  504300634
  537648201
  491504020
 2333424372

In [31]:
haskey(vprops, 446665330)

true